In [1]:
import random
from copy import deepcopy
import pickle
import pandas as pd
import numpy as np 
import itertools 
np.random.seed(0)

In [2]:
manifest = pd.read_pickle('human_azim_spotlight_stim_manifest.pdpkl')

In [14]:
manifest[manifest.cue_word ==]

,manifest_ix,orig_df_ix,client_id,corpus,gender,gender_int,sr,word,cue_client_id,cue_corpus,...,target_transcripts,target_f0,english_distractor_f0,mandarin_distractor_f0,dutch_distractor_f0,excerpt_distractor_src_fn,distractor_word,distractor_transcripts,exp_word_ix,word_int_label


In [4]:
manifest[manifest.word ]

,manifest_ix,orig_df_ix,client_id,corpus,gender,gender_int,sr,word,cue_client_id,cue_corpus,...,target_transcripts,target_f0,english_distractor_f0,mandarin_distractor_f0,dutch_distractor_f0,excerpt_distractor_src_fn,distractor_word,distractor_transcripts,exp_word_ix,word_int_label


In [28]:
## update rows to match 
manifest['sex_cond'] = manifest['sex_cond'].replace({"same":'same', 'diff':"different"})

In [29]:
manifest.columns

Index(['manifest_ix', 'orig_df_ix', 'client_id', 'corpus', 'gender',
       'gender_int', 'sr', 'word', 'cue_client_id', 'cue_corpus', 'cue_gender',
       'cue_gender_int', 'cue_sr', 'cue_total_file_duration_in_s', 'cue_word',
       'sex_cond', 'excerpt_src_fn', 'excerpt_cue_src_fn',
       'target_transcripts', 'target_f0', 'english_distractor_f0',
       'mandarin_distractor_f0', 'dutch_distractor_f0',
       'excerpt_distractor_src_fn', 'distractor_word',
       'distractor_transcripts', 'exp_word_ix', 'word_int_label'],
      dtype='object')

In [30]:
manifest['excerpt_distractor_src_fn'][0]

'/om/user/imgriff/datasets/human_word_rec_SWC_2024/sounds/distractor_excerpts/full_df_ix_0000_present-persian-poet-gal_eg_0.wav'

In [31]:
manifest[~manifest.client_id.str.contains('bowie|1906-cc|laurahale')].word.nunique()

488

In [32]:
manifest.shape

(976, 28)

In [33]:
manifest = manifest[~manifest.client_id.str.contains('bowie|1906-cc|laurahale')]

In [34]:
SNRs = [0]

def draw_block_spotlight(distractor_side, target_azim):
    distractor_deltas = [0, 10, 30, 90] # use same as azimuth 
    # snrs = [0, -3, -6, -9]
    # return tuples of (target_loc, distractor_locs, azim_delta, elev_delta. snr)
    trials = []
    for delta in distractor_deltas:
      # calc elevation delta relative to most extreme position
      if distractor_side == 'right':
        if target_azim == 40:
          target_azim = -40 # flip dir and move distractors to right 
        dist_azim = target_azim + delta 
      elif distractor_side == 'left':
        if target_azim == 40:
          target_azim = 40 # flip dir and move distractors to right 
        dist_azim = target_azim - delta 
        
      trial = ((target_azim, 0), (dist_azim, 0), delta, 0, 0)
      trials.append(trial)
    return trials
  

  

In [35]:
draw_block_spotlight('right', 0)

[((0, 0), (0, 0), 0, 0, 0),
 ((0, 0), (10, 0), 10, 0, 0),
 ((0, 0), (30, 0), 30, 0, 0),
 ((0, 0), (90, 0), 90, 0, 0)]

## Get trials locally

In [36]:
def sample_df(df, group, cond1, cond2, n):
	df_1 = df[df[f'{group}'] == cond1]
	df_2 = df[df[f'{group}'] == cond2]
	df_1_sample = df_1.sample(n=n)
	df_2_sample = df_2[~df_2.word.isin(df_1_sample.word)].sample(n=n)
    # keep original ixs to track metadata in analysis scripts 
	df_1_sample = df_1_sample.reset_index()
	df_1_sample.rename(columns={'index':'full_df_index'}, inplace=True)
	df_2_sample = df_2_sample.reset_index()
	df_2_sample.rename(columns={'index':'full_df_index'}, inplace=True)
	return pd.concat([df_1_sample, df_2_sample], axis=0, ignore_index=True)

def get_subset_df(df, n_words=480):
	n_to_samp = n_words // 4
	female_df = sample_df(df[df.gender == 'female'], 'sex_cond', 'same', 'different',n_to_samp)
	male_df = sample_df(df[(df.gender == 'male') & (~df.word.isin(female_df.word))], 'sex_cond', 'same', 'different', n_to_samp)
	return pd.concat([female_df, male_df], axis=0, ignore_index=True)

In [37]:
get_subset_df(manifest, n_words=12)

,full_df_index,manifest_ix,orig_df_ix,client_id,corpus,gender,gender_int,sr,word,cue_client_id,...,target_transcripts,target_f0,english_distractor_f0,mandarin_distractor_f0,dutch_distractor_f0,excerpt_distractor_src_fn,distractor_word,distractor_transcripts,exp_word_ix,word_int_label
0,253,126,711052,flyingtoaster,swc,female,0,44100,enough,flyingtoaster,...,"[enough, argument]",191.770569,200.409866,201.077530,193.877029,/om/user/imgriff/datasets/human_word_rec_SWC_2...,world,"[germany, its, world, war, i]",126,217
1,249,124,715126,lily5lace,swc,female,0,44100,england,lily5lace,...,"[coast, of, southeast, england, from, the, net...",197.918762,213.058731,213.515625,236.450577,/om/user/imgriff/datasets/human_word_rec_SWC_2...,power,"[electrical, power, plants, also]",124,215
2,222,111,621985,alanasings,swc,female,0,44100,dream,alanasings,...,"[an, american, dream, however]",155.831635,191.027786,214.342911,157.829956,/om/user/imgriff/datasets/human_word_rec_SWC_2...,between,"[angular, distance, between, the, two, stars]",111,199
3,712,356,605466,laura-s,swc,female,0,44100,search,laura-s,...,"[used, by, web, search, engines]",188.187607,187.600922,234.031067,175.450516,/om/user/imgriff/datasets/human_word_rec_SWC_2...,track,"[dance, pop, track, with, prominent, infor]",356,599
4,260,130,108943,popularoutcast,swc,female,0,44100,europe,popularoutcast,...,"[from, europe, asia, and, the, world]",164.452408,211.608185,194.840775,220.993149,/om/user/imgriff/datasets/human_word_rec_SWC_2...,failed,"[tank, failed]",130,224
5,752,376,140434,popularoutcast,swc,female,0,44100,since,popularoutcast,...,"[have, penetrated, since, one thousand, nine h...",167.056137,212.448517,192.774490,174.720169,/om/user/imgriff/datasets/human_word_rec_SWC_2...,seven,"[one hundred and thirty, seven, years, after, it]",376,624
6,65,520,973389,batwoodman,swc,male,1,44100,beach,batwoodman,...,"[and, the, beach, boys]",160.419525,137.524323,112.557663,135.696640,/om/user/imgriff/datasets/human_word_rec_SWC_2...,problem,"[and, the, cattle, problem, have, entered]",32,56
7,839,907,590740,dmitry-brant,swc,male,1,44100,theory,dmitry-brant,...,"[as, the, big, bang, theory, of, the, origin, ...",105.369301,119.462944,129.478745,126.554985,/om/user/imgriff/datasets/human_word_rec_SWC_2...,through,"[directly, through, the, temporal, bone]",419,699
8,514,745,854858,madridteacher1,swc,male,1,44100,money,madridteacher1,...,"[seeking, to, raise, money, for, the, global, ...",157.544144,218.597580,211.385498,166.435440,/om/user/imgriff/datasets/human_word_rec_SWC_2...,chief,"[from, the, bench, chief, justice, berger, sug...",257,426
9,236,606,509103,athf1234,swc,male,1,48000,effect,athf1234,...,"[clearly, to, little, effect]",115.288170,104.017044,139.393188,146.259323,/om/user/imgriff/datasets/human_word_rec_SWC_2...,short,"[dislike, cutting, short, his, rounded, duff]",118,208


In [38]:
from pathlib import Path
from IPython.display import display, Audio

stim_dir = Path("spotlight_sounds")
target_dir = stim_dir / "target_excerpts"
cue_dir = stim_dir / 'cue_excerpts'
dist_dir = stim_dir / 'distractor_excerpts'

In [39]:
manifest.shape

(956, 28)

In [40]:
np.random.seed(0)
eg_manifest = get_subset_df(manifest, 16)
print(len(eg_manifest))

ix = 2
target_path = target_dir / eg_manifest['excerpt_src_fn'].iloc[ix].split("/")[-1]
print(' '.join(eg_manifest['target_transcripts'].iloc[ix]))
print(eg_manifest['word'].iloc[ix])
display(Audio(target_path))
dist_path = dist_dir / eg_manifest['excerpt_distractor_src_fn'].iloc[ix].split("/")[-1]
print(' '.join(eg_manifest['distractor_transcripts'].iloc[ix]))
display(Audio(dist_path))


16
an american dream however
dream


angular distance between the two stars


In [41]:
from pathlib import Path

In [42]:
def create_experiment(distractor_side, num_trials_per_condition=20):
    stim_dir = Path("spotlight_sounds")
    
    target_dir = stim_dir / "target_excerpts"
    cue_dir = stim_dir / 'cue_excerpts'
    dist_dir = stim_dir / 'distractor_excerpts'

    all_front_target_trials = []
    all_fourty_target_trials = []
    all_ninety_target_trials = []
    n_total_trials = 0
    
    for i in range(num_trials_per_condition):
        front_trials = draw_block_spotlight(distractor_side, 0)
        fourty_target_trials = draw_block_spotlight(distractor_side, 40)
        ninety_target_trials = draw_block_spotlight(distractor_side, 0) ## will rotate participant 
        n_total_trials += len(front_trials)
        n_total_trials += len(fourty_target_trials)
        n_total_trials += len(ninety_target_trials)
        
        random.shuffle(front_trials)
        random.shuffle(fourty_target_trials)
        random.shuffle(ninety_target_trials)
        
        all_front_target_trials.append((front_trials, 'front_target')) 
        all_fourty_target_trials.append((fourty_target_trials, '40_target')) 
        all_ninety_target_trials.append((ninety_target_trials, '90_target'))
 
    random.shuffle(all_front_target_trials)
    random.shuffle(all_fourty_target_trials)
    random.shuffle(all_ninety_target_trials)
    experiment = all_front_target_trials + all_fourty_target_trials + all_ninety_target_trials

    print(f"Generating {n_total_trials} trials")
    full_df = pd.read_pickle('human_azim_spotlight_stim_manifest.pdpkl')
    full_df['sex_cond'] = full_df['sex_cond'].replace({"same":'same', 'diff':"different"})

    # explicitly screen examples with miss-matched talkers 
    full_df = full_df[~full_df.client_id.str.contains('bowie|1906-cc|laurahale')]# cull examples that made it through screening 
    print(len(full_df))
    participant_trial_stim_df = get_subset_df(full_df, n_words=n_total_trials).sample(frac=1.0).reset_index(drop=True)
    ## just need trial indices to get audio. Will match full_df_index string to ix number from participant_trial_df
    print(len(participant_trial_stim_df))
    i = 0
    array_manifest = []

    for (block, task) in experiment:
        for j, trial in enumerate(block):
            trial_idx = i
            target_src_fn = [str(target_dir / participant_trial_stim_df['excerpt_src_fn'].iloc[trial_idx].split("/")[-1])]
            cue_src_fn = [str(cue_dir / participant_trial_stim_df['excerpt_cue_src_fn'].iloc[trial_idx].split("/")[-1])]
            distractor_src_fn =  str(dist_dir / participant_trial_stim_df['excerpt_distractor_src_fn'].iloc[trial_idx].split("/")[-1])
            distractor_src_fn = [distractor_src_fn,]
            trial_word = participant_trial_stim_df.loc[trial_idx, 'word']
            distractor_word = participant_trial_stim_df.loc[trial_idx, 'distractor_word']
            distractor_word = distractor_word[0]
            block[j] = block[j] + (trial_word, trial_idx, distractor_word)
            array_manifest.append((trial[0], trial[1], trial[4], cue_src_fn, target_src_fn, distractor_src_fn))
            i += 1

    experiment_data = dict()
    experiment_data[f'block_front'] = dict()
    experiment_data[f'block_fourty'] = dict()
    experiment_data[f'block_ninety'] = dict()
    front_ix = 0 
    fourty_ix = 0
    ninety_ix = 0
    for i, (block, task) in enumerate(experiment):
        for j, trial in enumerate(block):
            trial_dict = {'target_loc': trial[0],
                                            'distractor_loc': trial[1],
                                            'azim_delta': trial[2],
                                            'elev_delta': trial[3],
                                            'snr': trial[4],
                                            'target_word': trial[5],
                                            'distractor_word': trial[7]}
                                            
            if task == 'front_target':
                experiment_data[f'block_front'][f'trial_{front_ix}'] = trial_dict
                front_ix += 1 
            
            elif task == '40_target':
                experiment_data[f'block_fourty'][f'trial_{fourty_ix}'] = trial_dict
                fourty_ix += 1 
                
            elif task == '90_target':
                experiment_data[f'block_ninety'][f'trial_{ninety_ix}'] = trial_dict
                ninety_ix += 1 

    trial_dict = {i:vals for i,vals in enumerate(array_manifest)}
    return experiment_data, array_manifest, trial_dict

In [43]:
experiment_data, array_manifest, trial_dict = create_experiment('left', 20)

Generating 240 trials
956
240


In [44]:
trial_ix = 30
experiment_data['block_front'][f'trial_{trial_ix}']['target_word'], array_manifest[trial_ix][4]

('radio',
 ['spotlight_sounds/target_excerpts/full_df_ix_0814_radio-radio-edit.wav'])

In [45]:
for block, trials in experiment_data.items():
    print(f"{len(trials)} in {block}")

80 in block_front
80 in block_fourty
80 in block_ninety


In [46]:
for block, trials in experiment_data.items():
    elev_deltas = [vals['azim_delta'] for vals in trials.values() ]
    target_locs = [vals['target_loc'][0] for vals in trials.values() ]
    dist_locs = [vals['distractor_loc'][0] for vals in trials.values() ]

    print(block)
    print("\t", np.unique(elev_deltas, return_counts=True))       
    print("\t", np.unique(target_locs, return_counts=True))       
    print("\t", np.unique(dist_locs, return_counts=True))       

block_front
	 (array([ 0, 10, 30, 90]), array([20, 20, 20, 20]))
	 (array([0]), array([80]))
	 (array([-90, -30, -10,   0]), array([20, 20, 20, 20]))
block_fourty
	 (array([ 0, 10, 30, 90]), array([20, 20, 20, 20]))
	 (array([40]), array([80]))
	 (array([-50,  10,  30,  40]), array([20, 20, 20, 20]))
block_ninety
	 (array([ 0, 10, 30, 90]), array([20, 20, 20, 20]))
	 (array([0]), array([80]))
	 (array([-90, -30, -10,   0]), array([20, 20, 20, 20]))


# Make new manifest here 

In [48]:
# write out manifests 
import pickle 
from pathlib import Path
# get n files in output dir 

# manifest to expmt stim 

# Name of sub directory to save experiment results - should match dir of trial dicts!
EXP_TYPE = "spotlight_v00" 
N_PER_CONDITION = 20  # 20 per elev x delta - can over elevation position in analysis 

out_dir = Path(f'speaker_array_manifests/{EXP_TYPE}')
out_dir.mkdir(exist_ok=True, parents=True)
n_files = len(list(out_dir.glob('*manifest.pkl')))

np.random.seed(1) # change seed for each participant!!!! 

if n_files % 2 == 0:
    distractor_side = 'left'
else:
    distractor_side = 'right'
print(distractor_side)

experiment, array_manifest, trial_dict = create_experiment(distractor_side, N_PER_CONDITION)

print(f"{sum([len(experiment[key]) for key in experiment.keys()])} total trials created")

###########################################################
# Enter the participant name from the threshold experiment
###########################################################

PART_NAME = "participant_001"  # 

# PART_NAME = f"participant_{n_files+1:03d}"
print(PART_NAME)

with open(out_dir / f'{PART_NAME}_meta.pkl', 'wb') as f:
    pickle.dump(experiment, f)

with open(out_dir / f'{PART_NAME}_array_manifest.pkl', 'wb') as f:
    pickle.dump(array_manifest, f)

with open(out_dir / f'{PART_NAME}_trial_dict.pkl', 'wb') as f:
    pickle.dump(trial_dict, f)

# get target key list 
key_out_path = Path(f"spotlight_expmt_keys")
key_out_path.mkdir(exist_ok=True, parents=True)

word_key = [trial['target_word'] for block in experiment.values() for trial in block.values()]

# save as json 
import json 
with open(key_out_path / f"{EXP_TYPE}_{PART_NAME}_key.json", "w") as f:
    json.dump(word_key, f)


right
Generating 240 trials
956
240
240 total trials created
participant_001


In [ ]:
### Run from above here ###

In [197]:
## confirm words are in dictionary 
dictionary = set([
    "about",
    "above",
    "access",
    "according",
    "across",
    "action",
    "active",
    "activities",
    "activity",
    "actually",
    "added",
    "addition",
    "additional",
    "africa",
    "after",
    "again",
    "against",
    "aircraft",
    "airport",
    "album",
    "alchemist",
    "allowed",
    "almost",
    "alone",
    "along",
    "already",
    "although",
    "always",
    "america",
    "american",
    "among",
    "ancient",
    "animals",
    "another",
    "answered",
    "anything",
    "appear",
    "appearance",
    "appeared",
    "appears",
    "appointed",
    "areas",
    "around",
    "artist",
    "asked",
    "associated",
    "association",
    "attack",
    "attended",
    "australia",
    "author",
    "available",
    "award",
    "awards",
    "based",
    "battle",
    "beach",
    "became",
    "because",
    "become",
    "before",
    "began",
    "beginning",
    "behind",
    "being",
    "believe",
    "believed",
    "below",
    "better",
    "between",
    "birds",
    "black",
    "blood",
    "board",
    "books",
    "border",
    "bridge",
    "bright",
    "bring",
    "british",
    "brother",
    "brothers",
    "brought",
    "brown",
    "building",
    "buildings",
    "built",
    "buried",
    "business",
    "california",
    "called",
    "campaign",
    "cant",
    "canada",
    "canadian",
    "cannot",
    "capital",
    "career",
    "carried",
    "cases",
    "catholic",
    "cause",
    "caused",
    "cemetery",
    "center",
    "central",
    "certain",
    "change",
    "changed",
    "changes",
    "character",
    "characters",
    "charge",
    "charles",
    "chief",
    "child",
    "children",
    "china",
    "chinese",
    "christian",
    "church",
    "cities",
    "civil",
    "class",
    "clear",
    "close",
    "closed",
    "coach",
    "coast",
    "collection",
    "college",
    "color",
    "comes",
    "coming",
    "commercial",
    "common",
    "commonly",
    "community",
    "companies",
    "company",
    "complete",
    "completed",
    "completely",
    "complex",
    "computer",
    "concept",
    "considered",
    "consists",
    "construction",
    "contains",
    "continue",
    "continued",
    "continues",
    "control",
    "could",
    "couldnt",
    "council",
    "countries",
    "country",
    "county",
    "couple",
    "course",
    "court",
    "cover",
    "covered",
    "created",
    "creek",
    "cross",
    "culture",
    "current",
    "currently",
    "dance",
    "daughter",
    "david",
    "death",
    "decided",
    "degree",
    "department",
    "described",
    "desert",
    "design",
    "designed",
    "despite",
    "developed",
    "development",
    "didnt",
    "different",
    "difficult",
    "direction",
    "director",
    "distance",
    "district",
    "divided",
    "division",
    "doctor",
    "doesnt",
    "doing",
    "dont",
    "dream",
    "during",
    "earlier",
    "early",
    "earth",
    "eastern",
    "economy",
    "educated",
    "education",
    "effect",
    "eight",
    "either",
    "elected",
    "election",
    "ended",
    "engine",
    "england",
    "english",
    "enough",
    "entered",
    "entire",
    "entirely",
    "episode",
    "especially",
    "established",
    "europe",
    "european",
    "event",
    "events",
    "eventually",
    "every",
    "everyone",
    "everything",
    "evidence",
    "example",
    "except",
    "exist",
    "experience",
    "failed",
    "family",
    "famous",
    "father",
    "feature",
    "featured",
    "features",
    "female",
    "festival",
    "field",
    "fight",
    "films",
    "final",
    "finally",
    "finished",
    "firefox",
    "first",
    "flight",
    "floor",
    "followed",
    "following",
    "football",
    "force",
    "forced",
    "forces",
    "forest",
    "formed",
    "former",
    "forms",
    "forward",
    "found",
    "founded",
    "fourth",
    "france",
    "french",
    "frequently",
    "friend",
    "friends",
    "front",
    "function",
    "further",
    "future",
    "games",
    "garden",
    "general",
    "generally",
    "george",
    "german",
    "germany",
    "getting",
    "given",
    "gives",
    "going",
    "government",
    "governor",
    "great",
    "greek",
    "green",
    "ground",
    "group",
    "groups",
    "guitar",
    "hands",
    "happen",
    "happened",
    "happy",
    "having",
    "health",
    "heard",
    "heart",
    "heavy",
    "helped",
    "higher",
    "highly",
    "himself",
    "historic",
    "historical",
    "history",
    "horse",
    "hospital",
    "hours",
    "house",
    "houses",
    "however",
    "human",
    "hundred",
    "husband",
    "immediately",
    "important",
    "include",
    "included",
    "includes",
    "including",
    "increased",
    "independent",
    "india",
    "indian",
    "industry",
    "influence",
    "information",
    "initially",
    "inside",
    "instead",
    "interest",
    "international",
    "internet",
    "involved",
    "island",
    "islands",
    "isnt",
    "issue",
    "issues",
    "italian",
    "itself",
    "james",
    "japan",
    "japanese",
    "joined",
    "junior",
    "killed",
    "known",
    "language",
    "languages",
    "large",
    "larger",
    "largest",
    "later",
    "leader",
    "leading",
    "league",
    "learn",
    "least",
    "leave",
    "leaves",
    "leaving",
    "lets",
    "level",
    "library",
    "light",
    "limited",
    "lines",
    "listed",
    "little",
    "lived",
    "lives",
    "living",
    "local",
    "located",
    "location",
    "london",
    "longer",
    "looked",
    "looking",
    "lower",
    "machine",
    "magazine",
    "mainly",
    "major",
    "majority",
    "makes",
    "making",
    "marked",
    "market",
    "marriage",
    "married",
    "material",
    "matter",
    "maybe",
    "meaning",
    "means",
    "media",
    "medical",
    "member",
    "members",
    "metal",
    "method",
    "michael",
    "middle",
    "might",
    "miles",
    "military",
    "minister",
    "minor",
    "minutes",
    "mixed",
    "model",
    "models",
    "modern",
    "moment",
    "money",
    "month",
    "months",
    "morning",
    "mostly",
    "mother",
    "mountain",
    "mouth",
    "moved",
    "movement",
    "movie",
    "multiple",
    "municipality",
    "museum",
    "music",
    "musical",
    "named",
    "names",
    "national",
    "native",
    "natural",
    "nature",
    "nearby",
    "needed",
    "needs",
    "network",
    "never",
    "newspaper",
    "night",
    "north",
    "northern",
    "notable",
    "noted",
    "nothing",
    "novel",
    "number",
    "numbers",
    "numerous",
    "offer",
    "offered",
    "offers",
    "office",
    "official",
    "often",
    "older",
    "opened",
    "operated",
    "operates",
    "operations",
    "order",
    "origin",
    "original",
    "originally",
    "other",
    "others",
    "outside",
    "owned",
    "paper",
    "parents",
    "paris",
    "parish",
    "particular",
    "particularly",
    "parts",
    "party",
    "passed",
    "people",
    "performance",
    "performed",
    "period",
    "person",
    "personal",
    "peter",
    "phone",
    "piece",
    "place",
    "placed",
    "places",
    "planet",
    "plans",
    "plant",
    "played",
    "player",
    "players",
    "playing",
    "plays",
    "please",
    "point",
    "police",
    "political",
    "politics",
    "popular",
    "population",
    "position",
    "possible",
    "power",
    "practice",
    "present",
    "president",
    "previous",
    "previously",
    "primarily",
    "primary",
    "private",
    "probably",
    "problem",
    "problems",
    "process",
    "produced",
    "production",
    "products",
    "professional",
    "program",
    "programs",
    "project",
    "property",
    "provide",
    "provided",
    "provides",
    "province",
    "public",
    "published",
    "quickly",
    "quite",
    "radio",
    "railroad",
    "railway",
    "raised",
    "range",
    "rather",
    "reached",
    "reading",
    "really",
    "reason",
    "received",
    "recent",
    "recently",
    "record",
    "recorded",
    "records",
    "referred",
    "region",
    "regional",
    "regular",
    "related",
    "relationship",
    "release",
    "released",
    "remained",
    "remains",
    "remember",
    "replaced",
    "required",
    "research",
    "result",
    "results",
    "return",
    "returned",
    "reviews",
    "richard",
    "right",
    "rights",
    "river",
    "robert",
    "roman",
    "round",
    "route",
    "royal",
    "running",
    "saint",
    "school",
    "schools",
    "science",
    "search",
    "season",
    "second",
    "section",
    "seemed",
    "separate",
    "series",
    "served",
    "serves",
    "service",
    "services",
    "seven",
    "several",
    "shall",
    "shape",
    "sheep",
    "short",
    "shortly",
    "should",
    "showed",
    "shows",
    "significant",
    "similar",
    "simple",
    "simply",
    "since",
    "single",
    "sister",
    "slightly",
    "slowly",
    "small",
    "smaller",
    "smith",
    "social",
    "society",
    "software",
    "something",
    "sometimes",
    "songs",
    "sound",
    "source",
    "south",
    "southern",
    "space",
    "spanish",
    "speak",
    "special",
    "species",
    "specific",
    "spent",
    "sports",
    "stage",
    "standard",
    "stars",
    "start",
    "started",
    "state",
    "states",
    "station",
    "stations",
    "still",
    "stone",
    "stood",
    "stopped",
    "stories",
    "story",
    "street",
    "strong",
    "structure",
    "student",
    "students",
    "studied",
    "studies",
    "studio",
    "study",
    "style",
    "subsequently",
    "success",
    "successful",
    "suddenly",
    "summer",
    "support",
    "supported",
    "surface",
    "surrounding",
    "system",
    "systems",
    "table",
    "taken",
    "takes",
    "taking",
    "talking",
    "taught",
    "teams",
    "technology",
    "television",
    "terms",
    "thats",
    "their",
    "themselves",
    "theory",
    "there",
    "theres",
    "therefore",
    "these",
    "theyre",
    "thing",
    "things",
    "think",
    "third",
    "thomas",
    "those",
    "though",
    "thought",
    "three",
    "through",
    "throughout",
    "times",
    "title",
    "today",
    "together",
    "towards",
    "tower",
    "township",
    "track",
    "tracks",
    "trade",
    "traditional",
    "train",
    "training",
    "trains",
    "travel",
    "trees",
    "tried",
    "trust",
    "trying",
    "turned",
    "twenty",
    "twice",
    "types",
    "typically",
    "under",
    "understand",
    "understood",
    "union",
    "unique",
    "united",
    "university",
    "unknown",
    "until",
    "upper",
    "using",
    "usually",
    "valley",
    "variety",
    "various",
    "version",
    "video",
    "village",
    "virginia",
    "voice",
    "waiting",
    "wanted",
    "wasnt",
    "water",
    "were",
    "website",
    "western",
    "whats",
    "where",
    "whether",
    "which",
    "while",
    "white",
    "whole",
    "widely",
    "william",
    "windows",
    "winter",
    "within",
    "without",
    "woman",
    "women",
    "wont",
    "words",
    "worked",
    "working",
    "works",
    "world",
    "would",
    "write",
    "writer",
    "writing",
    "written",
    "wrote",
    "years",
    "yellow",
    "youll",
    "youre",
    "young",
    "younger"
])

In [201]:
len(set(manifest.word.unique()).intersection(dictionary))/manifest.word.nunique()

1.0